# ❤️ Azure AI Agents

## Model Context Protocol (MCP) lab
![flow](../../utils/model-context-protocol.gif)

Playground to experiment the [Model Context Protocol](https://modelcontextprotocol.io/).

This lab includes the following MCP servers:
- Basic weather service: provide tools to get cities for a given country and retrieve random weather information for a specified city.

<a id='testconnection'></a>
### 🧪 Test the connection to the MCP servers and List Tools

In [13]:
import os, json, asyncio, time, requests
from mcp import ClientSession
from mcp.client.sse import sse_client
import nest_asyncio
nest_asyncio.apply()
from dotenv import load_dotenv
from datetime import datetime as pydatetime

load_dotenv()

server_url = "https://" + os.getenv("WEATHER_CONTAINERAPP_URL") + "/sse"

async def list_tools(server_url, authorization_header = None):
    headers = {"Authorization": authorization_header} if authorization_header else None
    async with sse_client(server_url, headers) as streams:
        async with ClientSession(streams[0], streams[1]) as session:
            await session.initialize()

            response = await session.list_tools()
            tools = response.tools
    print(f"✅ Connected to server {server_url}")
    print("⚙️ Tools:")
    for tool in tools:
        print(f"  - {tool.name}")
        print(f"     Input Schema: {tool.inputSchema}")
    
asyncio.run(list_tools(f"{server_url}"))

  + Exception Group Traceback (most recent call last):
  |   File "c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\Bancolombia\gbbai-ai-foundry-sdk-workshop\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3549, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |   File "C:\Users\pablocastao\AppData\Local\Temp\ipykernel_27212\59124045.py", line 27, in <module>
  |     asyncio.run(list_tools(f"{server_url}"))
  |   File "c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\Bancolombia\gbbai-ai-foundry-sdk-workshop\.venv\Lib\site-packages\nest_asyncio.py", line 30, in run
  |     return loop.run_until_complete(task)
  |            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\Bancolombia\gbbai-ai-foundry-sdk-workshop\.venv\Lib\site-packages\nest_asyncio.py", line 98, in run_until_complete
  |     return f.result()
  |            ^^^^^^^^^^
  |   File "C:\Users\pablocastao\AppData\Local\Progra

## Create Client and Load Azure AI Foundry

In [10]:
# Azure AI Projects
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    AgentEventHandler,
    RunStep,
    RunStepDeltaChunk,
    ThreadMessage,
    ThreadRun,
    MessageDeltaChunk,
    BingGroundingTool,
    FilePurpose,
    FileSearchTool,
    FunctionTool,
    ToolSet
)

credential = DefaultAzureCredential()
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

## Set Up MCP Tools

In [ ]:
#  Tool Call 
async def call_tool(mcp_session, function_name, function_args):
    try:
        func_response = await mcp_session.call_tool(function_name, function_args)
        func_response_content = func_response.content
    except Exception as e:
        func_response_content = json.dumps({"error": str(e)})
    return str(func_response_content)

# Get Tool List
async with sse_client(server_url) as streams:
        async with ClientSession(streams[0], streams[1]) as session:
            await session.initialize()
            response = await session.list_tools()
            tools = response.tools

# Convert to openai tool format
openai_tools = [{
                    "type": "function",
                    "function": {
                        "name": tool.name,
                        "parameters": tool.inputSchema
                    },
                } for tool in tools
]



  + Exception Group Traceback (most recent call last):
  |   File "c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\Bancolombia\gbbai-ai-foundry-sdk-workshop\.venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3547, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "C:\Users\pablocastao\AppData\Local\Temp\ipykernel_27212\518087778.py", line 9, in <module>
  |     async with sse_client(server_url) as streams:
  |   File "C:\Users\pablocastao\AppData\Local\Programs\Python\Python311\Lib\contextlib.py", line 204, in __aenter__
  |     return await anext(self.gen)
  |            ^^^^^^^^^^^^^^^^^^^^^
  |   File "c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\Bancolombia\gbbai-ai-foundry-sdk-workshop\.venv\Lib\site-packages\mcp\client\sse.py", line 43, in sse_client
  |     async with anyio.create_task_group() as tg:
  |   File "c:\Users\pablocastao\OneDrive - Microsoft\Work\Clients\Bancolombia\gbbai-ai-foundry-sdk-workshop\.venv

In [ ]:
# Create agent with OpenApi tool and process assistant run
model_name = os.getenv("MODEL_DEPLOYMENT_NAME")
instructions = (
    "You are a helpful enterprise assistant at Contoso. "
    f"Today's date is {pydatetime.now().strftime('%A, %b %d, %Y, %I:%M %p')}. "
    "You have access to tools to get information about the weather, "
)

prompt_content = "What's the current weather in Medellin?"

with project_client:
    agent = project_client.agents.create_agent(
        model=os.getenv("MODEL_DEPLOYMENT_NAME"),
        name="my-assistant",
        instructions=instructions,
        tools=openai_tools
    )
    utils.print_ok(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    utils.print_ok(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(thread_id=thread.id,
        role="user",
        content=prompt_content)
    utils.print_ok(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    utils.print_ok(f"Run finished with status: {run.status}")

    if run.status == "failed":
        utils.print_error(f"Run failed: {run.last_error}")

    # Print steps and function/tool details
    run_steps = project_client.agents.list_run_steps(thread_id=thread.id, run_id=run.id)
    for step in reversed(run_steps.data):
        utils.print_ok(f"Step {step['id']} status: {step['status']}")
        step_details = step.get("step_details", {})
        tool_calls = step_details.get("tool_calls", [])
        if tool_calls:
            for call in tool_calls:
                function_details = call.get("function", {})
                if function_details:
                    utils.print_info(f"Function details: {function_details}")

    project_client.agents.delete_agent(agent.id)

    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"🗨️ {messages.data[0].content[0].text.value}")

NameError: name 'openai_tools' is not defined